In [1]:
%matplotlib inline
import os, mne, glob, natsort, pdb
import numpy as np
import xarray as xr
import pandas as pd
mne.set_log_level('error')
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
def mne_apply(func, raw, verbose="WARNING"):
    """
    Apply function to data of `mne.io.RawArray`.
    From braindecode toolbox: https://github.com/robintibor/braindecode/blob/master/braindecode/mne_ext/signalproc.py#L75-L93
    
    Parameters
    ----------
    func: function
        Should accept 2d-array (channels x time) and return modified 2d-array
    raw: `mne.io.RawArray`
    verbose: bool
        Whether to log creation of new `mne.io.RawArray`.
    Returns
    -------
    transformed_set: Copy of `raw` with data transformed by given function.
    """
    new_data = func(raw.get_data())
    return mne.io.RawArray(new_data, raw.info, verbose=verbose)

In [3]:
lp = '.../'
sp = '.../'
tlims = [-2,2] # seconds
tlims_handpos = [0,4] # seconds
filt_freqs = [1,None] # Hz (low, high cutoffs)
n_splits = 4 # number of splits per subject
sbj_id = 'S15'
n_chans = 61 # number of EEG channels
event_dict = {'move':0x600,'rest':0x606}
# event labels: elbow flexion (0x600), elbow extension (0x601), supination (0x602),
#               pronation (0x603), hand close (0x604), hand open (0x605),
#               rest (0x606)
sfreq_new = 250 # Hz

In [4]:
# Create txt file with electrode MNI positions (mirrored to left hemisphere)
create_chan_pos_file = False

if create_chan_pos_file:
    chan_pos = pd.read_csv(lp+'roi_proj/eeg_elec_mni_pos.csv',index_col=0)
    ch_names = ['F3','F1','Fz','F2','F4',
                'FFC5h','FFC3h','FFC1h','FFC2h','FFC4h','FFC6h',
                'FC5','FC3','FC1','FCz','FC2','FC4','FC6','FTT7h',
                'FCC5h','FCC3h','FCC1h','FCC2h','FCC4h','FCC6h','FTT8h',
                'C5','C3','C1','Cz','C2','C4','C6','TTP7h',
                'CCP5h','CCP3h','CCP1h','CCP2h','CCP4h','CCP6h','TTP8h',
                'CP5','CP3','CP1','CPz','CP2','CP4','CP6',
                'CPP5h','CPP3h','CPP1h','CPP2h','CPP4h','CPP6h',
                'P3','P1','Pz','P2','P4','PPO1h','PPO2h']

    chan_locs = np.zeros([len(ch_names),3])
    for s,chan in enumerate(ch_names):
        curr_pos = chan_pos.loc[chan].values
        # Mirror to left hemisphere
#         if curr_pos[0] > 0:
#             curr_pos[0] = -curr_pos[0]
        chan_locs[s,:] = curr_pos
    
    chan_info = pd.DataFrame(chan_locs,columns=['X','Y','Z'])
    chan_info.to_csv(lp+'roi_proj/eeg_elec_mni_pos_bothH.txt', header=None, index=None, na_rep='nan')

In [5]:
natsort.natsorted(glob.glob(lp+sbj_id+'_ME/*.gdf'))

[]

In [6]:
natsort.natsorted(glob.glob(lp+sbj_id+'_ME/*.gdf'))

'.../S15'

In [7]:
if not os.path.exists(sp):
    os.mkdir(sp)

fnames_all = natsort.natsorted(glob.glob(lp+sbj_id+'_ME/*.gdf'))

for s,fname_curr in enumerate(fnames_all):
    print(fname_curr)
    # Load datafile
    dat_load = mne.io.read_raw_edf(fname_curr,preload=True)
    dat_hand_pos = dat_load.copy()
    
    ch_labels = dat_load.info['ch_names']
    dat = dat_load.drop_channels(ch_labels[n_chans:])
    assert len(dat.ch_names) == n_chans
    
    # Convert to millvolt for numerical stability of next operations
    dat = mne_apply(lambda a: a * 1e6, dat)
    
    # Common average reference
    dat.set_eeg_reference(ref_channels='average')
    
    # High-pass filter
    dat.filter(filt_freqs[0], filt_freqs[1])
    
    # Find events (769, 770, 771, 772)
    events,ev_dic_orig = mne.events_from_annotations(dat_load)
    ev_dic_orig[str(int(event_dict['move']))]
    
    # Epoch data around events
    event_id = {'rest': ev_dic_orig[str(int(event_dict['rest']))],
                'move': ev_dic_orig[str(int(event_dict['move']))]}
    
    drop_chan_pos = [val for val in ch_labels if val not in ['handPosX', 'handPosY', 'handPosZ']]
    dat_hand_pos.drop_channels(drop_chan_pos)
    dat_hand_pos._data[0,:] = np.sqrt(np.square(dat_hand_pos._data).sum(axis=0))
    dat_hand_pos.drop_channels(['handPosY', 'handPosZ'])
    ep_hand_pos = mne.Epochs(dat_hand_pos, events, event_id, tlims_handpos[0],
                             tlims_handpos[1], baseline=None, preload=True)
#     plt.plot(ep_hand_pos['move']._data[0,0,:].squeeze())
#     plt.show()
    move_ev_inds = np.nonzero(events[:,2]==event_id['move'])[0]
    print(events[move_ev_inds,0])
    for i in range(ep_hand_pos['move']._data.shape[0]):
        curr_trace = ep_hand_pos['move']._data[i,...].squeeze()
        curr_trace = np.abs(curr_trace-curr_trace[0])
        thresh=min(curr_trace.max()*.75,1)
        events[move_ev_inds[i],0] += np.nonzero(curr_trace>thresh)[0][0]
    print(events[move_ev_inds,0])
    
    if s==0:
        epochs = mne.Epochs(dat, events, event_id, tlims[0], tlims[1], baseline=None, preload=True)
    else:
        epochs_tmp = mne.Epochs(dat, events, event_id, tlims[0], tlims[1], baseline=None, preload=True)
        epochs = mne.concatenate_epochs([epochs,epochs_tmp])
print(epochs._data.shape[0])

# Resample epochs to match ECoG inputs
epochs.resample(sfreq_new)
    
# Add labels to data
event_id_labs = list(event_id.keys())
days_start = (np.arange(n_splits)+1).tolist()
recording_day,labels = [],[]
for i,lab_curr in enumerate(event_id_labs):
    ep_tmp = epochs[lab_curr]
    n_tmp = int(ep_tmp._data.shape[0])//n_splits
    days_curr = np.asarray(days_start * n_tmp)
    np.random.shuffle(days_curr)
    recording_day.extend(days_curr.tolist()) 
    if i==0:
        ecog_dat_sbj = ep_tmp.get_data().copy()
    else:
        ecog_dat_sbj = np.concatenate((ecog_dat_sbj,ep_tmp.get_data().copy()),axis=0)
    labels.extend([i+1]*ep_tmp.get_data().shape[0])

# Add labels to ECoG data
labels_arr = np.tile(np.expand_dims(np.asarray(labels),1),(1,ecog_dat_sbj.shape[2]))
labels_arr = np.expand_dims(labels_arr,1)
ecog_dat_sbj = np.concatenate((ecog_dat_sbj,labels_arr),axis=1)

# Convert to xarray and save
da_ecog = xr.DataArray(ecog_dat_sbj,
                  [('events', recording_day),
                   ('channels', np.arange(ecog_dat_sbj.shape[1])),
                   ('time', epochs.times)])
pdb.set_trace()
da_ecog.to_netcdf(sp+sbj_id+'_ecog_data.nc')
    
print('Sampling rate: '+str(epochs.info['sfreq'])+' Hz')

NameError: name 'epochs' is not defined